# M2608.001300 기계학습 기초 및 전기정보 응용<br> Assignment 4: Hidden Markov Model (HMM)

In [13]:
# Code from Chapter 16 of Machine Learning: An Algorithmic Perspective (2nd Edition)
# by Stephen Marsland (http://stephenmonika.net)

# You are free to use, change, or redistribute the code in any way you wish for
# non-commercial purposes, but please maintain the name of the original author.
# This code comes with no warranty of any kind.

# Stephen Marsland, 2008, 2014

# A basic Hidden Markov Model

## Setup
Check that Python 3.5 or later is installed (although Python 2.x may work, it is deprecated so we strongly recommend you use Python 3 instead).

In [14]:
# Python >=3.5 is required
import sys
assert sys.version_info >= (3, 5)

# Common imports
import numpy as np
from pandas import DataFrame

# to make this notebook's output stable across runs
np.random.seed(4)

## Problem 1. Evaluation problem
Get the probability of seeing an observation. <br>
We can use the following 2 algorithms:
<br>
* <font color=blue>*Forward algorithm*</font> (dynamic programming)
* <font color=blue>*Backward algorithm*</font> (dynamic programming)

In [81]:
scaling = True

In [82]:
"""
Let's denote the parameters of HMM model as,

pi : initial state probability
a : transition probability
b : emission probability
obs : observation sequence
"""
def HMMfwd(pi, a, b, obs):
    # TODO: Forward algorithm
    ####################### YOUR CODE HERE #######################
    nStates = np.shape(b)[0]
    T = np.shape(obs)[0]

    alpha = np.zeros((nStates,T))

    alpha[:,0] = pi*b[:,obs[0]]

    for t in range(1,T):
        for s in range(nStates):
            alpha[s,t] = b[s,obs[t]] * np.sum(alpha[:,t-1] * a[:,s])

    c = np.ones((T))
    if scaling:
        for t in range(T):
            c[t] = np.sum(alpha[:,t])
            alpha[:,t] /= c[t]
##############################################################
    return alpha,c

def HMMbwd(a, b, obs, c):
# TODO: Backward algorithm
####################### YOUR CODE HERE #######################
    nStates = np.shape(b)[0]
    T = np.shape(obs)[0]

    beta = np.zeros((nStates,T))

    beta[:,T-1] = 1.0 #aLast

    for t in range(T-2,-1,-1):
        for s in range(nStates):
            beta[s,t] = np.sum(b[:,obs[t+1]] * beta[:,t+1] * a[s,:])

    for t in range(T):
        beta[:,t] /= c[t]
    #beta[:,0] = b[:,obs[0]] * np.sum(beta[:,1] * pi)
##############################################################
    return beta

## Problem 2. Decoding problem
Get the underlying state sequence of an observation. <br>
We can use the <font color=blue>*Viterbi algorithm*</font> (dynamic programming).

In [75]:
def Viterbi(pi, a, b, obs):
# TODO: Viterbi algorithm
####################### YOUR CODE HERE #######################
    nStates = np.shape(b)[0]
    T = np.shape(obs)[0]

    path = np.zeros(T)
    delta = np.zeros((nStates,T))
    phi = np.zeros((nStates,T))

    delta[:,0] = pi * b[:,obs[0]]
    phi[:,0] = 0

    for t in range(1,T):
        for s in range(nStates):
            delta[s,t] = np.max(delta[:,t-1]*a[:,s])*b[s,obs[t]]
            phi[s,t] = np.argmax(delta[:,t-1]*a[:,s])

    path[T-1] = np.argmax(delta[:,T-1])
    for t in range(T-2,-1,-1):
        path[t] = phi[ int(path[t+1]) , t+1 ]
##############################################################
    return path, delta, phi

## Problem 3. Learning problem
Get the best parameters for the model. <br>
We can use the following 3 algorithms:
* *MLE* (maximum likelihood estimation)
* *Viterbi training* (different from Viterbi decoding)
* *Baum Welch algorithm* (a.k.a. forward-backward algorithm, uses expectation-maximization)

Here we implement the <font color=blue>*Baum Welch algorithm*</font>.

In [87]:
def BaumWelch(obs,nStates):

    T = np.shape(obs)[0]
    xi = np.zeros((nStates,nStates,T))

	# Initialise pi, a, b randomly
    pi = 1./nStates*np.ones((nStates))
    a = np.random.rand(nStates,nStates)
    b = np.random.rand(nStates,np.max(obs)+1)

    tol = 1e-5
    error = tol+1
    maxits = 100
    nits = 0

	# TODO: Baum Welch algorithm
	####################### YOUR CODE HERE #######################
    while ((error > tol) & (nits < maxits)):
        nits += 1
        oldpi = pi.copy()
        olda = a.copy()
        oldb = b.copy()

		# E step
        alpha,c = HMMfwd(pi,a,b,obs)
        beta = HMMbwd(a,b,obs,c) 

        for t in range(T-1):
            for i in range(nStates):
                for j in range(nStates):
                    xi[i,j,t] = alpha[i,t]*a[i,j]*b[j,obs[t+1]]*beta[j,t+1]
            xi[:,:,t] /= np.sum(xi[:,:,t])

# The last step has no b, beta in
        for i in range(nStates):
            for j in range(nStates):
                xi[i,j,T-1] = alpha[i,T-1]*a[i,j]
        xi[:,:,T-1] /= np.sum(xi[:,:,T-1])

# M step
        for i in range(nStates):
            pi[i] = np.sum(xi[i,:,0])
            for j in range(nStates):
                a[i,j] = np.sum(xi[i,j,:T-1])/np.sum(xi[i,:,:T-1])

            for k in range(max(obs)):
                found = (obs==k).nonzero()
                b[i,k] = np.sum(xi[i,:,found])/np.sum(xi[i,:,:])

        error = (np.abs(a-olda)).max() + (np.abs(b-oldb)).max() 
        print (nits, error, 1./np.sum(1./c), np.sum(alpha[:,T-1]))
##############################################################
    return pi, a, b	

## Toy examples
Here are the parameters for HMM model.

In [63]:
transition_data = {'state': ['TV', 'Bar', 'Party', 'Study'],
                          'TV'    : [0.4, 0.3, 0.1, 0.2],
                          'Bar'   : [0.6, 0.05, 0.1, 0.25],
                          'Party' : [0.7, 0.05, 0.05, 0.2],
                          'Study' : [0.3, 0.4, 0.25, 0.05]}

emission_data = {'observation': ['tired', 'hangover', 'anxiety', 'good'],
                        'TV'    : [0.2, 0.1, 0.2, 0.5],
                        'Bar'   : [0.4, 0.2, 0.1, 0.3],
                        'Party' : [0.3, 0.4, 0.2, 0.1],
                        'Study' : [0.3, 0.05, 0.3, 0.35]}

transition_probability = DataFrame(transition_data, columns=['state', 'TV', 'Bar', 'Party', 'Study'])
emission_probability = DataFrame(emission_data, columns=['observation', 'TV', 'Bar', 'Party', 'Study'])

In [64]:
transition_probability

,state,TV,Bar,Party,Study
0,TV,0.4,0.60,0.70,0.30
1,Bar,0.3,0.05,0.05,0.40
2,Party,0.1,0.10,0.05,0.25
3,Study,0.2,0.25,0.20,0.05


In [65]:
emission_probability

,observation,TV,Bar,Party,Study
0,tired,0.2,0.4,0.3,0.30
1,hangover,0.1,0.2,0.4,0.05
2,anxiety,0.2,0.1,0.2,0.30
3,good,0.5,0.3,0.1,0.35


In [66]:
try:
    a = transition_probability.drop(['state'], axis=1).to_numpy()
    b = emission_probability.drop(['observation'], axis=1).to_numpy()
except:
    a = transition_probability.drop(['state'], axis=1).values
    b = emission_probability.drop(['observation'], axis=1).values

In [67]:
# Assume the initial state probabilites are all equal to 0.25
pi = np.array([0.25, 0.25, 0.25, 0.25])
# obs = np.array(['tired', 'tired', 'good', 'hangover', 'hangover', 'anxiety', 'hangover', 'good'])
obs = np.array([0, 0, 3, 1, 1, 2, 1, 3])

## Problem 4-1. 
What is the probability of seeing current observation?

In [85]:
####################### YOUR CODE HERE #######################
scaling=False
HMMfwd(pi, a, b, obs)
##############################################################

(array([[5.00000000e-02, 1.15000000e-02, 3.68250000e-03, 1.12596250e-03,
         3.41476188e-04, 7.43701845e-05, 2.94586717e-05, 6.33472193e-06],
        [2.50000000e-02, 6.75000000e-03, 6.99062500e-04, 7.11837500e-04,
         1.87900484e-04, 1.12186492e-04, 1.23085622e-05, 1.25889706e-06],
        [5.00000000e-02, 1.27500000e-02, 4.02000000e-03, 3.54389063e-04,
         9.95800391e-05, 5.96778696e-05, 6.49272183e-06, 7.96546940e-06],
        [1.25000000e-01, 2.18750000e-02, 3.65093750e-03, 7.71576562e-04,
         2.24909953e-04, 2.13743557e-05, 2.49521511e-05, 5.82113505e-06]]),
 array([1., 1., 1., 1., 1., 1., 1., 1.]))

## Problem 4-2.
Given the current observation, what is the most likely sequence of states?

In [76]:
####################### YOUR CODE HERE #######################
Viterbi(pi, a, b, obs)
##############################################################

(array([3., 0., 0., 0., 0., 1., 0., 2.]),
 array([[5.000000e-02, 5.000000e-03, 6.000000e-04, 9.600000e-05,
         1.536000e-05, 1.843200e-06, 4.423680e-07, 5.308416e-08],
        [2.500000e-02, 3.125000e-03, 1.500000e-04, 7.200000e-05,
         1.152000e-05, 3.686400e-06, 2.211840e-07, 1.327104e-08],
        [5.000000e-02, 7.000000e-03, 1.050000e-03, 4.200000e-05,
         6.720000e-06, 2.150400e-06, 1.290240e-07, 9.289728e-08],
        [1.250000e-01, 7.500000e-03, 6.125000e-04, 7.875000e-05,
         8.640000e-06, 4.608000e-07, 4.423680e-07, 4.644864e-08]]),
 array([[0., 3., 0., 0., 0., 0., 1., 0.],
        [0., 3., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 2., 2., 0., 0., 1., 0.]]))

## Problem 4-3.
Given the current observation and the model, find the model parameters (transition probability, emission probability, initial state probability) that best fits the data.

In [89]:
####################### YOUR CODE HERE #######################
print (BaumWelch(obs,4))
##############################################################

1 1.523408176959474 0.125 0.39770339022815376
2 0.24509492979220976 0.125 0.0007244166427056073
3 0.23558760549466345 0.125 0.0015822825393684203
4 0.1756422781692704 0.125 0.0035284597478076355
5 0.15315211579097263 0.125 0.00566746424883069
6 0.152295169200435 0.125 0.0074477125655139035
7 0.14139363714161746 0.125 0.009886339271641419
8 0.15106928937733277 0.125 0.013939247874280076
9 0.12597518735893132 0.125 0.019264839213262958
10 0.09431618716325407 0.125 0.023713686561463845
11 0.07220410044384862 0.125 0.025972207120956506
12 0.05261859661248314 0.125 0.026847782956552824
13 0.036872649175507144 0.125 0.027210826392959256
14 0.02566862876587328 0.125 0.02739532217769465
15 0.017947990480269405 0.125 0.02750526563247221
16 0.012612657555556642 0.125 0.027575784692146
17 0.008893432612419705 0.125 0.027622185335238494
18 0.006281134144080621 0.125 0.027652868762905052
19 0.004436264862387737 0.125 0.027673032685832107
20 0.0031286513144593853 0.125 0.027686068912471568
21 0.0021